In [1]:
import os
import pandas as pd
from tqdm import tqdm
from classes.vqa_model_predictor import DefaultVqaModelPredictor
from evaluate.VqaMedEvaluatorBase import VqaMedEvaluatorBase
from common.utils import VerboseTimer
import vqa_logger 
import logging
from pathlib import Path
import datetime
logger = logging.getLogger(__name__)

Using TensorFlow backend.


In [2]:
%%capture
mp = DefaultVqaModelPredictor.get_contender()

datasets = {'test':mp.df_test, 'validation':mp.df_validation}
df_name_to_predict = 'test'


predictions = {}

for name, df in datasets.items():
    with VerboseTimer(f"Predictions for VQA contender {name}"):
        df_predictions = mp.predict(df)
        predictions[name] = df_predictions



predictions['validation'][:5]

[2019-04-02 18:04:55][common.utils][DEBUG] Starting 'Loading  VQA contender'
[2019-04-02 18:04:55][classes.vqa_model_predictor][DEBUG] Getting model #5 (post_concat_dense_units: 8;
Optimizer: RMSprop
loss: cosine_proximity
activation: sigmoid
prediction vector: answers
lstm_units: 0
batch_size: 64

model_location = 'C:\Users\Public\Documents\Data\2019\models\20190223_2239_45\vqa_model.h5')
[2019-04-02 18:04:55][common.utils][DEBUG] Starting 'Loading Model'
[2019-04-02 18:04:59][common.utils][DEBUG] Loading Model: 0:00:04.092725
[2019-04-02 18:04:59][classes.vqa_model_predictor][DEBUG] Getting model #72 (prediction_vector: words
 question_category: Abnormality
augmentations: 20 
epochs: 10
batch size: 64
based on model 5)
[2019-04-02 18:04:59][common.utils][DEBUG] Starting 'Loading Model'
[2019-04-02 18:05:01][common.utils][DEBUG] Loading Model: 0:00:01.889324
[2019-04-02 18:05:01][root][DEBUG] For Abnormality, got specialized model (DB: 72, Folder: ModelFolder(folder="C:\\Users\\Public

In [3]:
outputs = {}
for name, df_predictions in predictions.items():
    curr_predictions = df_predictions.prediction.values
    df_predicted = datasets[name]
    df_output = df_predicted.copy()
    df_output['image_id'] = df_output.path.apply(lambda p: p.rsplit(os.sep)[-1].rsplit('.', 1)[0])
    df_output['prediction'] = curr_predictions

    columns_to_remove = ['path',  'answer_embedding', 'question_embedding', 'group', 'diagnosis', 'processed_answer']
    for col in columns_to_remove:
        del df_output[col]

    sort_columns = sorted(df_output.columns, key=lambda c: c not in ['question', 'prediction', 'answer'])
    df_output = df_output[sort_columns]    
    outputs[name] = df_output

In [4]:
df_output_test = outputs['test']
df_output_validation = outputs['validation']

In [12]:
display = df_output_validation[df_output_validation.question_category == 'Abnormality']
display = df_output_validation
display.sample(10)

,question,answer,prediction,image_name,processed_question,question_category,image_id
14649,what is abnormal in the x-ray?,avascular necrosis,fracture stress disease bone osteochondritis,synpic32632.jpg,what is abnormal in the ray,Abnormality,synpic32632
14166,what organ systems can be evaluated with this ...,"lung, mediastinum, pleura",lung mediastinum pleura,synpic33102.jpg,what organ systems can be evaluated with this ray,Organ,synpic33102
14275,what organ system is being imaged?,skull and contents,skull and contents,synpic50410.jpg,what organ system is being imaged,Organ,synpic50410
13045,how was the image taken?,us - ultrasound,us ultrasound,synpic38739.jpg,how was the image taken,Modality,synpic38739
14734,what abnormality is seen in the image?,hiatal hernia,fracture disease dysplasia stress multiple,synpic25880.jpg,what abnormality is seen in the image,Abnormality,synpic25880
14308,what is abnormal in the ct scan?,"liposarcoma, myxoid liposarcoma",meningioma multiforme glioblastoma medulloblas...,synpic38894.jpg,what is abnormal in the ct scan,Abnormality,synpic38894
14054,what is the organ principally shown in this ul...,genitourinary,gastrointestinal,synpic43604.jpg,what is the organ principally shown in this ul...,Organ,synpic43604
14628,what is most alarming about this mri?,chronic lymphocytic leukemia (cll),spinal spine cyst epidural stenosis,synpic46993.jpg,what is most alarming about this mri,Abnormality,synpic46993
14130,what organ system is pictured here?,skull and contents,skull and contents,synpic58993.jpg,what organ system is pictured here,Organ,synpic58993
13607,which plane is this image in?,axial,axial,synpic28733.jpg,which plane is this image in,Plane,synpic28733


In [6]:
mp

VqaModelPredictor(model=5, specialized_classifiers={'Abnormality': 72, 'Modality': 69, 'Organ': 70, 'Plane': 71})

In [7]:
len(df_output_test), len(df_output_test.image_id.drop_duplicates())

(500, 500)

In [8]:
def get_str(df):
    strs = []
    debug_output_rows = df.apply(lambda row: row.image_id + '|'+ row.question + '|'+ row.prediction, axis=1 )
    output_rows = df.apply(lambda row: row.image_id + '|'+ row.prediction + '|'+row.answer, axis=1 )
    output_rows = output_rows.str.strip('|')
    rows = output_rows.values
    res = '\n'.join(rows)
    return res

res = get_str(df_output_test)
#res_val = get_str(df_output_validation)


In [9]:
print(res[:200])

synpic54082|cta ct angiography
synpic48556|no
synpic50696|iv
synpic37194|us doppler ultrasound
synpic31308|an angiogram
synpic24739|t1
synpic32365|yes
synpic47936|iv
synpic29792|yes
synpic43561|no
syn


### Get evaluations per category:

In [12]:
evaluations = {}
pbar = tqdm(df_output_validation.groupby('question_category'))
for question_category, df in pbar:        
    pbar.set_description(f'evaluating {len(df)} for {question_category} items')
    curr_predictions = df.prediction.values
    curr_ground_truth = df.answer.values
    curr_evaluations = VqaMedEvaluatorBase.get_all_evaluation(predictions=curr_predictions, ground_truth=curr_ground_truth)
    evaluations[question_category] = curr_evaluations    

   




  0%|                                                                                                                                                                                                                   | 0/4 [00:00<?, ?it/s]

evaluating 500 for Abnormality items:   0%|                                                                                                                                                                             | 0/4 [00:00<?, ?it/s]

evaluating 500 for Abnormality items:  25%|█████████████████████████████████████████▎                                                                                                                           | 1/4 [00:10<00:31, 10.57s/it]

evaluating 500 for Modality items:  25%|██████████████████████████████████████████                                                                                                                              | 1/4 [00:10<00:31, 10.57s/it]

evaluating 500 for Modality items:  50

### Get Total Evaluation:

In [13]:
total_evaluations = VqaMedEvaluatorBase.get_all_evaluation(predictions=df_output_validation.prediction.values, ground_truth=df_output_validation.answer.values)    
evaluations['Total'] = total_evaluations

In [14]:
evaluations
df_evaluations = pd.DataFrame(evaluations).T#.sort_values(by=('bleu'))
df_evaluations['sort'] = df_evaluations.index == 'Total'
df_evaluations = df_evaluations.sort_values(by = ['sort', 'wbss'])
del df_evaluations['sort']
df_evaluations

,bleu,wbss
Abnormality,0.166595,0.152170
Organ,0.708409,0.699596
Modality,0.763459,0.713040
Plane,0.739682,0.743336
Total,0.594536,0.577035


In [15]:
model_repr = repr(mp)
model_repr
sub_models = {category: folder for category, (model, folder) in mp.model_by_question_category.items()}
sub_models_str = '\n'.join([str(f'{category}: {folder} ({folder.prediction_data_name})') for category, folder in sub_models.items()])

model_description_copy = df_evaluations.copy()

def get_prediction_vector(category):
    if category in sub_models:
        return sub_models[category].prediction_data_name
    else:
        return '--'
    
model_description_copy['prediction_vector'] = model_description_copy.index.map(get_prediction_vector)


model_description =f'''
==Model==
{model_repr}

==Submodels==
{sub_models_str}

==validation evaluation==
{model_description_copy.to_string()}
'''


print(model_description)




==Model==
VqaModelPredictor(model=5, specialized_classifiers={'Abnormality': 72, 'Modality': 69, 'Organ': 70, 'Plane': 71})

==Submodels==
Abnormality: ModelFolder(folder="C:\\Users\\Public\\Documents\\Data\\2019\\models\\20190329_0854_26") (words)
Modality: ModelFolder(folder="C:\\Users\\Public\\Documents\\Data\\2019\\models\\20190329_0133_06") (answers)
Organ: ModelFolder(folder="C:\\Users\\Public\\Documents\\Data\\2019\\models\\20190329_0440_18") (answers)
Plane: ModelFolder(folder="C:\\Users\\Public\\Documents\\Data\\2019\\models\\20190329_0647_26") (answers)

==validation evaluation==
                 bleu      wbss prediction_vector
Abnormality  0.166595  0.152170             words
Organ        0.708409  0.699596           answers
Modality     0.763459  0.713040           answers
Plane        0.739682  0.743336           answers
Total        0.594536  0.577035                --



In [16]:
import time
now = time.time()
ts = datetime.datetime.fromtimestamp(now).strftime('%Y%m%d_%H%M_%S')
submission_base_folder = Path('C:\\Users\\Public\\Documents\\Data\\2019\\submissions')
submission_folder = submission_base_folder/ts
submission_folder.mkdir()


txt_path = submission_folder/f'submission_{ts}.txt'
txt_path.write_text(res)


model_description_path = submission_folder/f'model_description.txt'
model_description_path.write_text(model_description)

with pd.HDFStore(str(submission_folder/ 'predictions.hdf')):
    for name, df_predictions in predictions.items():
        store[name] = df_predictions

NameError: name 'store' is not defined